This is just a concised part of the model, with a selection of the pretrained model to be used. The actual notebook is submitted at another location.

In [1]:
#https://github.com/microsoft/vscode-jupyter/wiki/Installing-Python-packages-in-Jupyter-Notebooks
%pip install tensorflow
%pip install datasets
%pip install torch
%pip install nltk
%pip install transformers

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable



You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# note: may take quite awhile to run
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch

In [3]:
# read csv and split train/test
df = pd.read_csv('../static/nlp/data.csv')
train_txt, test_txt, train_label, test_label = train_test_split(df['question'], df['label'], test_size=0.2, random_state=42)

# encode text
pretrained = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(pretrained)
train_t = tokenizer(train_txt.tolist(), truncation=True, padding=True)

labelencoder = LabelEncoder()
train_l = labelencoder.fit_transform(train_label.tolist())
test_l = labelencoder.fit_transform(test_label.tolist())

# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


# function for preparing the test dataset for prediction as well as customer's input qns. 
#train dataset don't need to have function as it would not have a new dataset such as the customer input in the chatbot
def preparing(testq, tokenizer):
    testq_list = testq.tolist()
    test_t = tokenizer(testq_list, truncation=True, padding=True)
    return test_t

train_dataset = Dataset(train_t, train_l.tolist())
test_dataset = Dataset(preparing(test_txt,tokenizer), test_l.tolist())

# load model
model = BertForSequenceClassification.from_pretrained(pretrained, num_labels=len(np.unique(train_l)))

# training
training_args = TrainingArguments("test_trainer", num_train_epochs=10)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
# getting metric such as accuracy of model
# huggingface's metric was used to check the model's accuracy and other scores during training
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    logits, labels = pred
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [5]:
from transformers import Trainer
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset
)
trainer.train()

# eg if customer ask in chatbot, start by tokenizing the question first
#customer = np.array(['Duration of my plans covering me'])
#qns = Dataset(preparing(customer,tokenizer))
#ans = trainer.predict(qns).predictions.argmax(1)
#result = labelencoder.inverse_transform(ans)
#print("The label is:", result[0])

***** Running training *****
  Num examples = 2930
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3670
 14%|█▎        | 500/3670 [20:41<2:15:47,  2.57s/it]Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json


{'loss': 0.9411, 'learning_rate': 4.3188010899182565e-05, 'epoch': 1.36}


Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin
 27%|██▋       | 1000/3670 [42:36<1:53:25,  2.55s/it]Saving model checkpoint to test_trainer\checkpoint-1000
Configuration saved in test_trainer\checkpoint-1000\config.json


{'loss': 0.0758, 'learning_rate': 3.637602179836512e-05, 'epoch': 2.72}


Model weights saved in test_trainer\checkpoint-1000\pytorch_model.bin
 41%|████      | 1500/3670 [1:03:25<1:25:05,  2.35s/it]Saving model checkpoint to test_trainer\checkpoint-1500
Configuration saved in test_trainer\checkpoint-1500\config.json


...

Model weights saved in test_trainer\checkpoint-3000\pytorch_model.bin
 95%|█████████▌| 3500/3670 [2:32:26<07:38,  2.70s/it]Saving model checkpoint to test_trainer\checkpoint-3500
Configuration saved in test_trainer\checkpoint-3500\config.json


{'loss': 0.0006, 'learning_rate': 2.3160762942779292e-06, 'epoch': 9.54}


Model weights saved in test_trainer\checkpoint-3500\pytorch_model.bin
100%|██████████| 3670/3670 [2:40:07<00:00,  2.29s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 3670/3670 [2:40:07<00:00,  2.62s/it]

{'train_runtime': 9607.937, 'train_samples_per_second': 3.05, 'train_steps_per_second': 0.382, 'train_loss': 0.14555174255850203, 'epoch': 10.0}


TrainOutput(global_step=3670, training_loss=0.14555174255850203, metrics={'train_runtime': 9607.937, 'train_samples_per_second': 3.05, 'train_steps_per_second': 0.382, 'train_loss': 0.14555174255850203, 'epoch': 10.0})

In [6]:
# save the model to disk
#file might be too large to push

# Save to file in the static directory
file_location = "../static/nlp/example.pt"
torch.save(model, file_location)
ai_model = torch.load(file_location)

In [7]:
print('Has access to GPU: ', torch.cuda.is_available())

trainer = Trainer(
            model=ai_model
        )

customer = np.array(['insurier'])
qns = Dataset(preparing(customer,tokenizer))
ans = trainer.predict(qns).predictions.argmax(1)
result = labelencoder.inverse_transform(ans)
print("The label is:", result[0])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Has access to GPU:  False


***** Running Prediction *****
  Num examples = 1
  Batch size = 8
  0%|          | 0/1 [00:00<?, ?it/s]

The label is: insurer
